In [ ]:
!pip install flask
!pip install flask-ngrok
!pip install pyngrok

In [1]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return "Hello, world!"

# Run the app on port 5004
if __name__ == "__main__":
    app.run(port=5004)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5004/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Feb/2024 21:36:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2024 21:36:22] "GET /favicon.ico HTTP/1.1" 404 -


In [17]:
#PURPOSE AND PERCENTAGES
import os
import pandas as pd
from flask import Flask, request, render_template
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
import logging

# Set up logging
logging.basicConfig(filename='app.log', level=logging.DEBUG)

# Load your CSV data into a Pandas DataFrame
data = pd.read_csv('bibertrain.csv')

# Extract features (numerical frequencies) and target (communicative purpose)
X_train = data.iloc[:, 3:]  # Assuming feature columns start from the fourth column
y_train = LabelEncoder().fit_transform(data['main_full'])

# Create and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Create and fit a label encoder for inverse transformation
label_encoder = LabelEncoder()
label_encoder.fit(data['main_full'])

# Create and fit a scaler for scaling input data
scaler = StandardScaler()
scaler.fit(X_train)

app = Flask(__name__, template_folder='./')

@app.route('/', methods=['GET', 'POST'])
def predict_communicative_purpose():
    if request.method == 'POST':
        uploaded_file = request.files['file']
        if uploaded_file:
            try:
                input_data = pd.read_csv(uploaded_file)

                # Ensure the order of columns in the input data matches the training data
                input_data = input_data[X_train.columns]

                # Scale the input features using the same scaler
                input_data_scaled = scaler.transform(input_data)

                # Predict the communicative purpose based on the uploaded features
                prediction_probabilities = model.predict_proba(input_data_scaled)[0]

                # Get the top communicative purpose and its probability
                top_communicative_purpose_index = prediction_probabilities.argmax()
                top_communicative_purpose = label_encoder.inverse_transform([top_communicative_purpose_index])[0]
                top_probability = prediction_probabilities[top_communicative_purpose_index]

                # Get the percentages for all communicative purposes
                percentages = {label: probability * 100 for label, probability in zip(model.classes_, prediction_probabilities)}

                logging.debug("Input Data:")
                logging.debug(input_data)
                logging.debug(f"Top Communicative Purpose: {top_communicative_purpose}")
                logging.debug(f"Top Probability: {top_probability}")
                logging.debug(f"Percentages: {percentages}")

                return render_template('index.html', communicative_purpose=top_communicative_purpose, probability=top_probability, percentages=percentages)
            except Exception as e:
                logging.error(f"Prediction Error: {str(e)}")
                return render_template('index.html', error_message=str(e))
    
    return render_template('index.html', communicative_purpose=None, probability=None, percentages=None, error_message=None)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5006)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/ingridveloso/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
 * Running on all addresses.
 * Running on http://10.17.230.131:5006/ (Press CTRL+C to quit)
10.17.230.131 - - [06/Feb/2024 16:44:43] "GET / HTTP/1.1" 200 -
/Users/ingridveloso/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
10.17.230.131 - - [06/Feb/2024 16:44:49] "POST / HTTP/1.1" 200 -


In [ ]:
#MORE THAN ONE ROW IN A FILE
import os
import pandas as pd
from flask import Flask, request, render_template
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
import logging

# Set up logging
logging.basicConfig(filename='app.log', level=logging.DEBUG)

# Load the CSV data into a Pandas DataFrame
data = pd.read_csv('bibertrain.csv')

# Extract features (numerical frequencies) and target (communicative purpose)
X_train = data.iloc[:, 3:]  # Assuming feature columns start from the fourth column
y_train = LabelEncoder().fit_transform(data['main_full'])

# Create and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Create and fit a label encoder for inverse transformation
label_encoder = LabelEncoder()
label_encoder.fit(data['main_full'])

# Create and fit a scaler for scaling input data
scaler = StandardScaler()
scaler.fit(X_train)

# Initialize Flask app
app = Flask(__name__, template_folder='./')

# Define route for prediction
@app.route('/', methods=['GET', 'POST'])
def predict_communicative_purpose():
    if request.method == 'POST':
        uploaded_file = request.files['file']
        if uploaded_file:
            try:
                input_data = pd.read_csv(uploaded_file)

                # Ensure the order of columns in the input data matches the training data
                input_data = input_data[X_train.columns]

                # Scale the input features using the same scaler
                input_data_scaled = scaler.transform(input_data)

                # Make predictions for each row in the input data
                predictions = []
                for i in range(len(input_data_scaled)):
                    row_input = input_data_scaled[i].reshape(1, -1)
                    prediction_probabilities = model.predict_proba(row_input)[0]
                    top_communicative_purpose_index = prediction_probabilities.argmax()
                    top_communicative_purpose = label_encoder.inverse_transform([top_communicative_purpose_index])[0]
                    top_probability = prediction_probabilities[top_communicative_purpose_index]
                    percentages = {label: probability * 100 for label, probability in zip(model.classes_, prediction_probabilities)}
                    predictions.append({
                        'communicative_purpose': top_communicative_purpose,
                        'probability': top_probability,
                        'percentages': percentages
                    })

                logging.debug("Input Data:")
                logging.debug(input_data)
                logging.debug(f"Predictions: {predictions}")

                return render_template('index.html', predictions=predictions)
            except Exception as e:
                logging.error(f"Prediction Error: {str(e)}")
                return render_template('index.html', error_message=str(e))

    return render_template('index.html', predictions=None, error_message=None)

# Run the Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5007)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/ingridveloso/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
 * Running on all addresses.
 * Running on http://192.168.1.175:5007/ (Press CTRL+C to quit)
192.168.1.175 - - [09/Feb/2024 21:36:32] "GET / HTTP/1.1" 200 -
192.168.1.175 - - [09/Feb/2024 21:36:32] "GET /favicon.ico HTTP/1.1" 404 -
/Users/ingridveloso/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/ingridveloso/opt/anaconda